# Ejemplo búsquedas con k-d tree

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 08 de junio de 2025 

Usaremos la librería FLANN (Fast Library for Approximate Nearest Neighbors)
https://github.com/flann-lib/flann

Es una librería escrita en C++ que tiene un wrapper para python llamado PyFlann.

Instalar PyFlann `1.9.2` con:

```
# instalar con conda (no usar pip)
conda install pyflann 

```

**Nota: NO usar pip** porque instalará una versión antigua `1.6.14`.

Si se está usando anaconda, instalar con: `conda install -c conda-forge pyflann`


In [ ]:
# primero cargar algunas funciones auxiliares
import os
import numpy
import time
import pyflann


# calcular la calidad de la respuesta contra el resultado "ideal" dado por el linear scan
class Evaluador:
    def __init__(self, nns_ideal, distancias_ideal, tiempo_ideal):
        self.nns_ideal = nns_ideal
        self.distancias_ideal = distancias_ideal
        self.tiempo_ideal = tiempo_ideal
        self.correctas = 0
        self.incorrectas = 0
        self.tiempo = 0
        self.precision = 0
        self.eficiencia = 0

    def evaluar_busqueda(self, nns, distancias, segundos):
        self.correctas = 0
        self.incorrectas = 0
        for i in range(len(self.nns_ideal)):
            if nns[i] == self.nns_ideal[i] or distancias[i] == self.distancias_ideal[i]:
                self.correctas += 1
            else:
                self.incorrectas += 1
        self.tiempo = segundos
        self.precision = self.correctas / (self.correctas + self.incorrectas)
        self.eficiencia = self.tiempo / self.tiempo_ideal

# Ejemplo 1: Buscar en Dataset A

In [ ]:
# cargar un conjunto de vectores
datasetA_q = numpy.load("dataset_a_q.npy")
datasetA_r = numpy.load("dataset_a_r.npy")

print(
    "Dataset A: conjunto_Q={} conjunto_R={}".format(datasetA_q.shape, datasetA_r.shape)
)

### Búsqueda de Q en R usando linear scan

El resultado obtenido por linear scan (los vectores más cercanos) son usados como el modelo ideal para luego medir efectividad.

In [ ]:
# crear un objeto flann
flann = pyflann.FLANN()

# construir el indice para linear scan
flann.build_index(datasetA_r, algorithm="linear")

# ejecutar el linear scan
t0 = time.time()

print(
    "iniciando linear scan de Q={} vectores de consulta de dimensión {}, buscando en R={} vectores de dimension {} ...".format(
        datasetA_q.shape[0],
        datasetA_q.shape[1],
        datasetA_r.shape[0],
        datasetA_r.shape[1],
    )
)
nns_LS, dists_LS = flann.nn_index(datasetA_q, num_neighbors=1, cores=1)

segundos_LS = time.time() - t0

print("Linear scan en Dataset A toma {:.1f} segundos".format(segundos_LS))

# objeto para evaluar las busquedas aproximadas
evaluadorA = Evaluador(nns_LS, dists_LS, segundos_LS)

### Guardar resultados en disco

Se guarda una lista de largo Q, con el número del vector más parecido de R.  
(se usará más adelante en otro ejemplo)

In [ ]:
filename = "dataset_a-nns-linear_scan.npy"

numpy.save(filename, nns_LS)
print("resultados guardados en {}".format(filename))

print(nns_LS.shape)
print(nns_LS)
print(dists_LS)

### Comprobar que la búsqueda obtiene efectividad 100% en un tiempo de 100%

In [ ]:
# para probar, se evaluará la misma búsqueda exacta 
# obtendrá un 100% de correctas en un 100% del tiempo porque se está comparando consigo mismo
evaluadorA.evaluar_busqueda(nns_LS, dists_LS, segundos_LS)

print("Linear scan en dataset A:")
print(
    "  correctas={} incorrectas={} tiempo={:>.2f} segundos".format(
        evaluadorA.correctas, evaluadorA.incorrectas, evaluadorA.tiempo
    )
)
print(
    "  precision={:.1%} tiempo={:.1%}".format(
        evaluadorA.precision, evaluadorA.eficiencia
    )
)

### Construir un kd tree con el conjunto R

In [ ]:
# construir el indice kdtree
t0 = time.time()

flann.build_index(datasetA_r, algorithm="kdtree", trees=1)

segundos_construccion = time.time() - t0

print("construcción kd tree={:.2f} segundos".format(segundos_construccion))

### Búsqueda exacta de Q en R usando el kd tree

Notar que hay una ganancia pequeña en el tiempo de búsqueda.
Incluso a veces puede ser más lento que el linear scan.

In [ ]:
# busqueda exacta del 1-NN usando el ultimo indice construido (kd tree)
t0 = time.time()

nn, dist = flann.nn_index(datasetA_q, num_neighbors=1, cores=1, checks=-1)

segundos = time.time() - t0

evaluadorA.evaluar_busqueda(nn, dist, segundos)

print(
    "kd tree EXACTO  precision={:.1%}  tiempo={:.2f} seg. ({:.1%} del tiempo de Linear scan)".format(
        evaluadorA.precision, evaluadorA.tiempo, evaluadorA.eficiencia
    )
)

### Búsqueda aproximada usando el k-d tree

El parámetro `checks` es la cantidad de hojas a visitar, lo que permite ajustar el tiempo de búsqueda y calidad de respuesta.

In [ ]:
# busqueda aproximada del 1-NN usando el ultimo indice construido (kd tree)
t0 = time.time()

# "checks" controla la velocidad de búsqueda y calidad de respuesta
nn, dist = flann.nn_index(datasetA_q, num_neighbors=1, cores=1, checks=1)

segundos = time.time() - t0

evaluadorA.evaluar_busqueda(nn, dist, segundos)

print(
    "kd tree APROX  precision={:6.1%} tiempo={:>5.2f} seg. ({:5.1f}% del tiempo de Linear scan)".format(
        evaluadorA.precision, evaluadorA.tiempo, evaluadorA.eficiencia
    )
)

### Secuencia de búsquedas usando distintos valores de aproximación

In [ ]:
# podemos hacer varios experimentos con el mismo índice, variando checks al buscar
for checks in (1, 10, 100, 1000, -1):
    t0 = time.time()
    nn, dist = flann.nn_index(datasetA_q, num_neighbors=1, cores=1, checks=checks)
    segundos = time.time() - t0

    evaluadorA.evaluar_busqueda(nn, dist, segundos)
    print(
        "kd tree checks={:>4}  precision={:>6.1%}  tiempo={:>5.2f} seg. ({:>5.1%} del tiempo de Linear Scan)".format(
            checks, evaluadorA.precision, evaluadorA.tiempo, evaluadorA.eficiencia
        )
    )

# Ejemplo 2: Buscar en Dataset B

### Primero usar linear scan para obtener el resultado ideal y el tiempo total

In [ ]:
# este es otro conjunto de vectores (notar la diferencia de performance del indice)
datasetB_q = numpy.load("dataset_b_q.npy")
datasetB_r = numpy.load("dataset_b_r.npy")

print(
    "Dataset B: conjunto_Q={} conjunto_R={}".format(datasetB_q.shape, datasetB_r.shape)
)

# construir el indice para linear scan para el nuevo dataset
flann.build_index(datasetB_r, algorithm="linear")

# ejecutar el linear scan
t0 = time.time()
nns_LS2, dists_LS2 = flann.nn_index(datasetB_q, num_neighbors=1, cores=1)
segundos_LS2 = time.time() - t0
print("Linear scan en Dataset B toma {:.1f} segundos".format(segundos_LS2))

# objeto para evaluar las busquedas aproximadas
evaluadorB = Evaluador(nns_LS2, dists_LS2, segundos_LS2)

### Crear un kd tree y hacer una serie de búsquedas con distintas aproximaciones

In [ ]:
# construir el indice kdtree
t0 = time.time()
flann.build_index(datasetB_r, algorithm="kdtree", trees=1)
segundos_construccion = time.time() - t0
print("construcción kd tree={:.2f} segundos".format(segundos_construccion))

for checks in (1, 10, 100, 1000, -1):
    t0 = time.time()
    nn, dist = flann.nn_index(datasetB_q, num_neighbors=1, cores=1, checks=checks)
    segundos = time.time() - t0

    evaluadorB.evaluar_busqueda(nn, dist, segundos)
    print(
        "kd tree checks={:>4}  precision={:>6.1%}  tiempo={:>5.2f} seg. ({:>5.1%} del tiempo de Linear Scan)".format(
            checks, evaluadorB.precision, evaluadorB.tiempo, evaluadorB.eficiencia
        )
    )

# Pregunta

Notar que los dos datasets tienen la misma cantidad de vectores de la misma dimensionalidad (Q=5.000 vectores, R=100.000 vectores, vectores de 128-d). Además, en todas las búsquedas se usó `cores=1` para evitar sobrecargas que pudieran influir en los tiempos obtenidos.

El Linear scan en ambos datasets demora el mismo tiempo. Sin embargo, los resultados de la búsqueda aproximada no son muy parecidos en ambos datasets. Por ejemplo, comparar la precision lograda y el tiempo que tomó en ambos datasets cuando se usó kd tree con `checks=1000`.

**¿A qué se puede deber que un dataset obtenga mejores aproximaciones que el otro?**